# 🧠 Erwartungswerte E(V) – Alles in einem Notebook
Dieses Notebook vereint **Modellbildung**, **Rekursion**, **Analyse aller Strategien**, **Visualisierung** und **Exportmöglichkeiten** in einem durchgehenden Format. Ideal für Unterricht, Selbststudium und Fortbildung.

## 🧮 Modellidee
Für eine Spielsituation $V = (v_1, \dots, v_m)$ gibt die rekursive Formel den Erwartungswert der verbleibenden Spielrunden an, um alle Chips abzuräumen:

$$
E(V) = 
\begin{cases}
0, & \text{falls } V = (0,\dots,0) \\
\frac{1}{s} + \sum_{j=1}^{m} \frac{p_j}{s} \cdot \mathbb{1}(v_j \ge 1) \cdot E(V_j), & \text{sonst}
\end{cases}
$$

**Dabei gilt:**
- $s = \sum\limits_{j=1}^{m} p_j \cdot \mathbb{1}(v_j \ge 1)$ ist die Gesamtwahrscheinlichkeit, ein besetztes Feld zu treffen  
- $V_j = (v_1, \dots, v_j - 1, \dots, v_m)$ ist die Spielsituation nach einem Treffer auf Feld $j$

## ⚙️ Setup

In [ ]:
from fractions import Fraction
from functools import lru_cache

## 🎯 Wahrscheinlichkeiten und Startzustand

In [ ]:
p = (Fraction(1, 2), Fraction(1, 3), Fraction(1, 6))
S = (3, 2, 1)

## 🔧 Hilfsfunktionen

In [ ]:
def is_terminal(state):
    return all(v == 0 for v in state)

def next_state(state, index):
    return state[:index] + (state[index] - 1,) + state[index + 1:]

## 🔁 Zentrale rekursive Funktion

In [ ]:
@lru_cache(maxsize=None)
def E(state):
    if is_terminal(state):
        return Fraction(0)
    s = sum(p[i] for i in range(len(state)) if state[i] > 0)
    res = Fraction(1, s)
    for i in range(len(state)):
        if state[i] > 0:
            res += (p[i] / s) * E(next_state(state, i))
    return res

## 📊 Beispielauswertung

In [ ]:
EX = E(S)
print(f"E({S}) = {EX} ({float(EX):.5})")

## 🧪 Alle Strategien analysieren

In [ ]:
from itertools import combinations_with_replacement

def all_distributions(n, m):
    raw = combinations_with_replacement(range(m), n)
    for comb in raw:
        counts = [0] * m
        for i in comb:
            counts[i] += 1
        yield tuple(counts)

results = []
for strat in all_distributions(sum(S), len(S)):
    ev = E(strat)
    results.append((strat, ev))

results.sort(key=lambda x: x[1])
for s, ev in results[:5]:
    print(f"E({s}) = {ev} ({float(ev):.3f})")

## 📈 Visualisierung aller Strategien

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator

states = [s for s, _ in results]
values = [float(ev) for _, ev in results]

plt.figure(figsize=(12, 5))
bars = plt.bar(range(len(values)), values, color='skyblue', width=0.8)

ax = plt.gca()
if len(states) <= 30:
    ax.set_xticks(range(len(states)))
    ax.set_xticklabels([str(s) for s in states], rotation=90, fontsize=9)
else:
    ax.xaxis.set_major_locator(MultipleLocator(int(len(states)/20)))
    ax.set_xticklabels([])

plt.xlabel("Strategien", fontsize=12)
plt.ylabel("E(V)", fontsize=12)
plt.title(f"Erwartungswerte für {sum(S)} Chips auf {len(S)} Feldern", fontsize=14)
plt.tight_layout()
# plt.savefig("strategien_plot.png", dpi=300, bbox_inches='tight')  # Optionaler Export
plt.show()